In [5]:
import json
import numpy as np
import pandas as pd
import redis
from IPython.display import display, HTML, clear_output
from ipywidgets import Dropdown, Button, Output

# Set pandas display option
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Redis connection info
redis_host = "rss.redis.cache.windows.net"
redis_port = 6379
redis_password = "AQouyZ83AaEt7lBhgpLCAhBLip3ygGHfZAzCaM3IaqI="
redis_db = 0
redis_retry_on_timeout = True
redis_socket_timeout = 5

planner_names = [
    "MTGTaskPlanner",
    "ALEFTaskPlanner",
    "MTGTaskPlannerWithGMM",
    "DynamicMTGTaskPlannerWithGMM",
    "DynamicMTGTaskPlannerWithGMM_V2",
    "DynamicMTGTaskPlannerWithGMM_25.0",
    "DynamicMTGTaskPlannerWithGMM_50.0",
    "DynamicMTGTaskPlannerWithGMM_75.0",
    "DynamicMTGTaskPlannerWithGMM_100.0",
    "MTGTaskPlannerWithAtlas",
    "DynamicMTGPlannerWithAtlas",
    "DynamicMTGPlannerWithAtlas_V2",
    "DynamicMTGPlannerWithAtlas_25.0",
    "DynamicMTGPlannerWithAtlas_50.0",
    "DynamicMTGPlannerWithAtlas_75.0",
    "DynamicMTGTaskPlannerWithAtlas_100.0",

]

# Connect to Redis
r = redis.StrictRedis(
    host=redis_host,
    port=redis_port,
    password=redis_password,
    db=redis_db,
    socket_timeout=redis_socket_timeout,
    retry_on_timeout=redis_retry_on_timeout
)

# Retrieve and display keys for user selection
keys = r.keys(pattern='*')
dropdown = Dropdown(options=keys)
button = Button(description="Load Data")
output = Output()

display(dropdown, button, output)

def load_data(b):
    with output:
        clear_output()
        key = dropdown.value
        raw_data = r.lrange(key, 0, -1)
        result_data = [json.loads(data) for data in raw_data]

        planner_stats = {
                            planner_name: {
                                'num' : 0, 
                                'num_success' : [], 
                                'motion_planning_time' : [],
                                'task_planning_time' : [],
                                'updating_time' : [],
                                'solution_length' : [],
                                'num_attempts' : [],
                                'total_planning_time' : [],
                                'set_start_and_goal_time' : [],
                            } 

                        for planner_name in planner_names
                        }

        for d in result_data:
            if d['planner_name'] in planner_stats:
                dic = planner_stats[d['planner_name']]
                dic['num'] += 1
                dic['num_success'].append(d['success'] == "true")
                if d['success'] == "true":
                    dic['motion_planning_time'].append(d['motion_planning_time'])
                    dic['task_planning_time'].append(d['task_planning_time'])
                    dic['total_planning_time'].append(d['total_planning_time'])
                    dic['updating_time'].append(d['updating_time'])
                    dic['solution_length'].append(d['solution_length'])
                    dic['num_attempts'].append(d['num_attempts'])
                    dic['set_start_and_goal_time'].append(d['set_start_and_goal_time'])

        output_dict = []
        for planner_name in planner_names:
            stats = planner_stats[planner_name]

            current_planner_eval_dict = {}
            current_planner_eval_dict["Planner Name"] = planner_name
            current_planner_eval_dict["Total Number of Tasks"] = len(stats['num_success'])
            current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
            current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
            current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )
            current_planner_eval_dict["Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
            current_planner_eval_dict["Set Start and Goal Time"] = np.mean(np.array(stats['set_start_and_goal_time']) )
            current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
            current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )
            current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
            output_dict.append(current_planner_eval_dict)
        df = pd.DataFrame(output_dict).set_index('Planner Name')

        display(df)
        
        

button.on_click(load_data)


Dropdown(options=(b'open_door_g_100_i_1434:1712011660.62_abcfe481-d910-4686-b179-f22e18f62fe3', b'slide_cup_g_…

Button(description='Load Data', style=ButtonStyle())

Output()